In [35]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [36]:
def get_SP500_stocks():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S.26P_500_Component_Stocks"
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html5lib")

    table = soup.find_all('table')[0]
    # Exclude header row
    rows = table.find_all('tr')[1:]
    #print(len(rows))
    #print(rows[len(rows)-1])
    stocks = []
    for row in rows:
        cols = row.find_all('td')
        stocks.append(cols[0].get_text())

    return stocks
sp500 = get_SP500_stocks() 

In [68]:

enddate = '2017-11-01'
news = []
for symbol in sp500:
    if symbol == 'AAPL':
        i = 0
        fl = 1
        while fl == 1:
            url = 'https://finance.google.com/finance/company_news?q=' + symbol +  \
                  '&enddate=' + enddate + '&start=' + str(i * 10) + '&num=10'
            x = requests.get(url)
            print i
            print url
            data = x.text
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup.prettify())
            quote = soup.find_all('a', {"href" : lambda L: L and L.startswith('http://news')})
            if quote:
                for row in quote:
                    try:
                        item = row.parent.findNext('span',attrs={'class': 'date'}).next_element
                    except:
                        item = news[-1][2]
                        #news.append((symbol,row.next_element,news[-1][1]))
                    #print item
                    if 'hours ago' in str(item):
                        item = datetime.now().date()
                    else:
                        if item <> news[-1][2]:
                            item = datetime.strptime(item.replace(',',''), '%b %d %Y').date()
                    news.append((symbol,row.next_element,item))
            else:
                fl = 0
            i = i + 1
print news 

0
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=0&num=10
1
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=10&num=10
2
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=20&num=10
3
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=30&num=10
4
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=40&num=10
5
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=50&num=10
6
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=60&num=10
7
https://finance.google.com/finance/company_news?q=AAPL&enddate=2017-11-01&start=70&num=10
[(u'AAPL', u"Apple, Inc. Could Bring TrueDepth and Face ID to Next Year's iPad\xa0Pros", datetime.date(2017, 10, 9)), (u'AAPL', u'\nApple Inc. (AAPL) iPhone Expectations Still Too High', datetime.date(2017, 10, 7)), (u'AAPL', u"\nHow the Next Apple Inc. iPhone Will

In [49]:
print news

[(u'AAPL', u"Apple, Inc. Could Bring TrueDepth and Face ID to Next Year's iPad\xa0Pros", datetime.date(2017, 10, 9)), (u'AAPL', u'\nApple Inc. (AAPL) iPhone Expectations Still Too High', datetime.date(2017, 10, 7)), (u'AAPL', u"\nHow the Next Apple Inc. iPhone Will Help Fill Intel Corp.'s Factories", datetime.date(2017, 10, 7)), (u'AAPL', u'How Apple Inc. Can Do a Mid-Cycle iPhone X\xa0Refresh', datetime.date(2017, 10, 7)), (u'AAPL', u'Apple, Inc. Earnings: Mark Your\xa0Calendar', datetime.date(2017, 10, 5)), (u'AAPL', u"\n1 Problem With Apple Inc.'s iPhone Chip Strategy", datetime.date(2017, 10, 9)), (u'AAPL', u'\nThe Broad Impact Of The iPhone X', datetime.date(2017, 10, 9)), (u'AAPL', u'3 Stocks to Watch on Monday: Apple Inc. (AAPL), Facebook Inc (FB) and\xa0...', datetime.date(2017, 10, 9)), (u'AAPL', u'\nBustling Stocks in Focus: Apple Inc. (NASDAQ:AAPL) and Microsoft Corporation ...', datetime.date(2017, 10, 9)), (u'AAPL', u"No, Apple Inc. Does Not Slow Down Your iPhone So You'll